<a href="https://colab.research.google.com/github/AtharvaDalvi2703/ML_NLP_worksheet1/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!jupyter nbconvert --to notebook --inplace \
    --ClearOutputPreprocessor.enabled=True \
    --ClearMetadataPreprocessor.enabled=True \
    "/content/sentiment_pipeline.ipynb"


!head -n5 "/content/sentiment_pipeline.ipynb"



[NbConvertApp] WARNING | pattern '/content/sentiment_pipeline.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=Tru

In [ ]:
%env WANDB_DISABLED=true
%pip install wandb

env: WANDB_DISABLED=true


In [ ]:
%pip install --upgrade transformers datasets
import transformers
%pip install transformers[torch]
%pip install 'accelerate>=0.26.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is i

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset=load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize(data):
  return tokenizer(data["text"], padding=True, truncation=True,max_length=256) #512

In [ ]:
tokenized_text=dataset.map(tokenize,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=preds, references=labels)["accuracy"],
        "f1":       f1_metric.compute(predictions=preds, references=labels, average="binary")["f1"],
    }

In [ ]:

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    fp16=True,
    num_train_epochs=1,
    report_to=[]
)


In [ ]:

accuracy_metric = evaluate.load("accuracy")
f1_metric       = evaluate.load("f1")


In [ ]:
small_train = tokenized_text["train"] \
    .shuffle(seed=42) \
    .select(range(5000))
small_test  = tokenized_text["test"] \
    .shuffle(seed=42) \
    .select(range(1000))

In [ ]:



from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset= small_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-12-4106634494.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.384100,0.331300,0.890000,0.889336


TrainOutput(global_step=625, training_loss=0.36792053833007815, metrics={'train_runtime': 15102.7762, 'train_samples_per_second': 0.331, 'train_steps_per_second': 0.041, 'total_flos': 657777638400000.0, 'train_loss': 0.36792053833007815, 'epoch': 1.0})

In [ ]:
eval_metrics = trainer.evaluate()
print(f"Eval Accuracy: {eval_metrics['eval_accuracy']:.4f}")
print(f"Eval F1:       {eval_metrics['eval_f1']:.4f}")


Eval Accuracy: 0.8900
Eval F1:       0.8893


In [ ]:
trainer.save_model("fine_tuned_bert_imdb")
tokenizer.save_pretrained("fine_tuned_bert_imdb")

('fine_tuned_bert_imdb/tokenizer_config.json',
 'fine_tuned_bert_imdb/special_tokens_map.json',
 'fine_tuned_bert_imdb/vocab.txt',
 'fine_tuned_bert_imdb/added_tokens.json',
 'fine_tuned_bert_imdb/tokenizer.json')

In [ ]:
from transformers import pipeline


sentiment = pipeline(
    "text-classification",
    model="fine_tuned_bert_imdb",
    tokenizer="fine_tuned_bert_imdb"
)


print(sentiment("What an amazing film—truly a masterpiece!"))



Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.9912369251251221}]


Pipeline Overview-

The machine learning pipeline launches by loading the IMDb movie‐review dataset through the Hugging Face datasets library, we have split the data into training and test sets and fixed a random seed for the sake of reproducibility.

Components and Rationale
Tokenizer-

We have used the method AutoTokenizer.from_pretrained("bert-base-uncased") to convert raw text into token IDs. We used padding and truncating with a maximum length of 256 tokens to get a balance between expressive capacity and computational costs.

Model-

We have used "AutoModelForSequenceClassification" (and a num_labels=2 argument) to provide a pre‐trained BERT encoder and lightweight classification head model. The selection of the model takes advantage of transfer learning to generalize better when trained with only a limited amount of observation data.

Training & Evaluation-

The pipeline also sets the learning rate (2e‑5), batch size (8), and to train one epoch (sufficient for proof of concept while managing GPU runtime). The Trainer handler optimises the model and includes logging & checkpointing capabilities. The pipeline's evaluate library also permits accuracy and F1‑score (binary average) to be computed.

Inference-

once the fine‑tuned model and tokenizer are saved we are able to wrap them in a Hugging Face pipeline for end‑to‑end text‑classification, providing ease of deployment and allowing rapid prototype capabilities.

Challenges faced-

The major challenge faced is the training phase, the training took significant time, and transformer also gave some issues in training arguments, so install some required libraries.
To make the training process fast I used small portion of the training and test dataset.
Anyway the worksheet gave me a great fun.